# CDO bindings test

Additional example can be found here: https://www.ecmwf.int/sites/default/files/elibrary/2018/18722-cdos-python-bindings.pdf

Before using the cdo bindigs we need to import the cdo bindings package (`python-cdo` in `conda` or `mamba`).
Additionally, before calling an operator, an object has to be created first.

We make use of the `tempdir` argument in order to store temporary data output in a custom folder and avoid heavy memory usage.

In [1]:
import cdo

tempPath = '/scratch/users/mnurisso/tempcdo/'
cdo = cdo.Cdo(tempdir=tempPath)

In [2]:
# Load some test data

path = '/work/datasets/obs/ERA5/mean_sea_level_pressure/mon/'
filename = 'ERA5_mean_sea_level_pressure_mon_0.25x0.25_sfc_1979-2021.nc'

infile = path+filename

## Simple command test

We try to run a simple `showname` cdo command that does not save any output file.

In [3]:
cdo.showname(input=infile)

['msl']

We try now to run a simple command having as output a file, to check how it is saved normally and how instead we can convert the output directly as a `xarray` `Dataset`

In [5]:
msl_ma = cdo.monmean(input=infile)
print(msl_ma)

/scratch/users/mnurisso/tempcdo/cdoPy66ebs_vc


In the first case the result of the cdo command is saved in a temporary file in the temporary folder that we have specified at the beginning. The `print()` command returns the path of this temporary file and cannot be analized with the `xarray` tools. It can be instead used as input for a new cdo command.

If we want to save the output as `xDataset` for further analysis:

In [6]:
msl_ma_dataset = cdo.monmean(input=infile,returnXDataset='msl_ma_dataset')
print(msl_ma_dataset)

<xarray.Dataset>
Dimensions:    (time: 516, bnds: 2, lon: 1440, lat: 721)
Coordinates:
  * time       (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2021-12-01
  * lon        (lon) float64 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
  * lat        (lat) float64 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75 -90.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    msl        (time, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 2.1.1 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    institution:  European Centre for Medium-Range Weather Forecasts
    history:      Tue Feb 14 15:51:06 2023: cdo -O -s -f nc -monmean /work/da...
    frequency:    mon
    CDO:          Climate Data Operators version 2.1.1 (https://mpimet.mpg.de...


In [11]:
cdo.sub(input=[msl_ma,msl_ma_av],returnXDataset='test')

Commands requiring multiple inputs file can be called as well:
`cdo.sub(input=[msl_ma,msl_ma_av])`

Commands can also be concatenated like `cdo.timmean(input= '-monmean' + infile)`

An output file can be specified and it can be returned by the cdo command:

`cdo.div(input=[sub_nao,std_ma],output=ofile)`

## Cleanup the temporary files

Temporary files are not deleted when the jupyter notebook is closed. 
In order to empty the space we need to run an additional command.

In [8]:
cdo.cleanTempDir()